In [ ]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import month,year,col,sum
from snowflake.snowpark.version import VERSION
from snowflake.core import Root
from snowflake.core.task import Task, StoredProcedureCall
from snowflake.core.task.dagv1 import DAG, DAGTask, DAGOperation
from snowflake.core import CreateMode

# Misc
from datetime import timedelta
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

session = get_active_session()
# Add a query tag to the session. This helps with monitoring and troubleshooting.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"aiml_notebooks_ad_spend_roi", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}


In [ ]:
snow_df_spend = session.sql(
"""
    SELECT *
    FROM CAMPAIGN_SPEND
""")
snow_df_spend.limit(10).collect()

In [ ]:
# Stats per Month per Channel
snow_df_spend_per_channel = session.sql(
"""
    SELECT 
        YEAR(DATE) AS YEAR,
        MONTH(DATE) AS MONTH,
        CHANNEL,
        SUM(TOTAL_COST) AS TOTAL_COST
    FROM CAMPAIGN_SPEND
    GROUP BY YEAR(DATE), MONTH(DATE), CHANNEL
    ORDER BY YEAR, MONTH
""")

snow_df_spend_per_channel.limit(10).collect()

In [ ]:
snow_df_spend_per_month = snow_df_spend_per_channel \
    .pivot('CHANNEL') \
    .sum('TOTAL_COST') \
    .sort('YEAR','MONTH')

snow_df_spend_per_month = snow_df_spend_per_month.select(
    col("YEAR"),
    col("MONTH"),
    col("'search_engine'").as_("SEARCH_ENGINE"),
    col("'social_media'").as_("SOCIAL_MEDIA"),
    col("'video'").as_("VIDEO"),
    col("'email'").as_("EMAIL")
)

snow_df_spend_per_month.limit(10).collect()